In [1]:
#import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk, spacy,re,gensim
import re


In [2]:
dataset=pd.read_csv('K8 Reviews v0.2.csv')

In [3]:
dataset['sentiment'].value_counts()

0    7712
1    6963
Name: sentiment, dtype: int64

In [4]:
dataset.shape

(14675, 2)

In [5]:
dataset.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [6]:
data_list=dataset['review'].str.lower()

In [7]:
data_list[14673]

'this phone is nice processing will be successful but dual camera is not be successfully in dual mod'

In [8]:
#word tokenization
from nltk.tokenize import word_tokenize

In [9]:
token_list=[word_tokenize(i) for i in data_list]

In [10]:
token_list[0]

['good', 'but', 'need', 'updates', 'and', 'improvements']

In [11]:
### removing stopwords 

from nltk.corpus import stopwords
stop_words= stopwords.words('english')
from string import punctuation
punlist=list(punctuation)
removal_list = stop_words+punlist
from nltk.stem import WordNetLemmatizer

In [12]:

finale=[]

for i in range(len(token_list)):
    newer=[]
    for word in token_list[i]:
        if word not in removal_list:
            newer.append(word)
    finale.append(newer)


In [13]:
len(finale)

14675

In [14]:
# POS tagging
tagged=[]
for i in finale :
        w=nltk.pos_tag(i)
        tagged.append(w)

In [15]:
print(tagged[0])

[('good', 'JJ'), ('need', 'NN'), ('updates', 'NNS'), ('improvements', 'NNS')]


In [16]:
# selecting only noun tokens
review_noun=[]
for sent in tagged:
    for token in sent:
        if re.search('NN.*',token[1]):
         review_noun.append(token[0])

In [17]:
review_noun[1]

'updates'

In [18]:
len(review_noun)

87894

In [19]:
# lemmatizing noun words

review_lemma=[]
lemm=WordNetLemmatizer()
for word in review_noun:
    review_lemma.append(lemm.lemmatize(word).split())

In [20]:
len(review_lemma)

87894

In [21]:
review_lemma[1]

['update']

In [22]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary 
import gensim.corpora as corpora

In [23]:
diction=gensim.corpora.Dictionary(review_lemma)

In [24]:
bow_corpus = [diction.doc2bow(doc) for doc in review_lemma]


In [25]:
model=gensim.models.ldamodel.LdaModel(corpus=bow_corpus, num_topics=6 ,id2word=diction,passes=8)

In [26]:
diction[[112][0]]

'day'

In [27]:
model.get_topic_terms(0)

[(20, 0.109619245),
 (29, 0.07836437),
 (28, 0.045426905),
 (149, 0.03132566),
 (70, 0.026158294),
 (57, 0.024702078),
 (48, 0.02366366),
 (287, 0.01965517),
 (192, 0.018962152),
 (277, 0.018015165)]

In [28]:
for idx, topic in model.print_topics(-1):
    print("Topic:{}, \nWords: {}".format(idx,topic))

Topic:0, 
Words: 0.110*"problem" + 0.078*"note" + 0.045*"k8" + 0.031*"device" + 0.026*"speaker" + 0.025*"range" + 0.024*"life" + 0.020*"support" + 0.019*"waste" + 0.018*"drain"
Topic:1, 
Words: 0.154*"product" + 0.065*"day" + 0.060*"mobile" + 0.035*"heating" + 0.033*"amazon" + 0.026*"work" + 0.020*"sound" + 0.018*".." + 0.015*"glass" + 0.015*"video"
Topic:2, 
Words: 0.427*"phone" + 0.083*"issue" + 0.055*"price" + 0.052*"performance" + 0.031*"month" + 0.016*"sim" + 0.015*"superb" + 0.015*"everything" + 0.015*"experience" + 0.012*"game"
Topic:3, 
Words: 0.097*"quality" + 0.068*"time" + 0.061*"lenovo" + 0.041*"money" + 0.037*"backup" + 0.035*"charge" + 0.035*"heat" + 0.034*"service" + 0.033*"screen" + 0.032*"hour"
Topic:4, 
Words: 0.222*"camera" + 0.057*"feature" + 0.040*"network" + 0.034*"call" + 0.024*"mode" + 0.024*"option" + 0.023*"buy" + 0.017*"update" + 0.016*"speed" + 0.016*"please"
Topic:5, 
Words: 0.210*"battery" + 0.035*"use" + 0.028*"charger" + 0.023*"processor" + 0.022*"return

In [29]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model, texts=review_lemma, dictionary=diction, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.8358863526755173


In [30]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(model, bow_corpus,diction)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.395536 -0.006406       1        1  17.728445
1      0.073949  0.061313       2        1  16.498920
3      0.076649  0.076822       3        1  16.494692
0      0.076808  0.069399       4        1  16.444323
5      0.081484  0.166615       5        1  16.430155
4      0.086645 -0.367742       6        1  16.403471, topic_info=        Term         Freq        Total Category  logprob  loglift
18     phone  6651.000000  6651.000000  Default  30.0000  30.0000
71    camera  3203.000000  3203.000000  Default  29.0000  29.0000
11   battery  3033.000000  3033.000000  Default  28.0000  28.0000
56   product  2233.000000  2233.000000  Default  27.0000  27.0000
20   problem  1584.000000  1584.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
382     till    92.898895    93.745293   Topic6  -5.0442   1.7986
113    worth    91.431160    92.277550   Topic6  -5.0601   1.7985
480   policy    90.097855    90.944252   Topic6  -5.0748   1.7983
150      con    83.372086    84.218475   Topic6  -5.1524   1.7976
265     sale    80.635895    81.482285   Topic6  -5.1857   1.7972

[210 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
63        2  0.996447        ..
9         2  0.998007    amazon
214       2  0.992532   android
466       1  0.988626  anything
702       2  0.994305       app
...     ...       ...       ...
541       4  0.992968    volume
192       4  0.997218     waste
672       5  0.995419      week
118       2  0.998089      work
113       6  0.986155     worth

[180 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 1, 6, 5])

In [47]:
wordlist=[]

for topic in model.get_topic_terms(1):
        wordlist.append(topic)

In [48]:
wordlist[0]

(56, 0.1540413)

In [33]:
df=pd.DataFrame(wordlist,index=None,columns=['Topics','Keywords'])

df=df.drop('Topics',axis=1)
df.insert(0,'Topics',('Features','Performance','Value for money','Agility ','Battery ','Camera display'))

In [34]:
df

,Topics,Keywords
0,Features,"0.110*""problem"" + 0.078*""note"" + 0.045*""k8"" + ..."
1,Performance,"0.154*""product"" + 0.065*""day"" + 0.060*""mobile""..."
2,Value for money,"0.427*""phone"" + 0.083*""issue"" + 0.055*""price"" ..."
3,Agility,"0.097*""quality"" + 0.068*""time"" + 0.061*""lenovo..."
4,Battery,"0.222*""camera"" + 0.057*""feature"" + 0.040*""netw..."
5,Camera display,"0.210*""battery"" + 0.035*""use"" + 0.028*""charger..."


In [35]:
df.to_csv('Topics_List.csv') 